In [236]:
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [260]:
hardcode = ['7/7', '7/7', '6/7 + Bonus', '6/7', '5/7', '5/7', '4/7', '4/7', '3/7 + Bonus']
size = [(7, False), (7, True), (6, True), (6, False), (5, False), (5, True), (4, False), (4, True), (3, True)]
intersections = dict(zip(size, hardcode))

filename = 'numbers.txt'


def get_url(year, month, day):
    url_template = 'https://lottery.olg.ca/en-ca/winning-numbers/lotto-max/winning-numbers?'
    year = str(year)
    startDate = year + '-' + '{:02d}'.format(month) + '-' + '{:02d}'.format(day - 1)
    endDate = year + '-' + '{:02d}'.format(month) + '-' + '{:02d}'.format(day + 1)
    url = url_template + 'startDate=' + startDate + '&' + 'endDate=' + endDate
    return url

def get_data(url):
    data = dict()
    http = urllib3.PoolManager()
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, 'html.parser')
    
    # getting numbers
    nums = soup.findAll('span', attrs={'class': 'winning-numbers-number'})
    win_numbers = []
    for number in nums:
        win_numbers.append(number.text)
    
    # getting money wins from the table
    table = soup.find_all('tbody')[0]
    values = dict()
    for row in table.find_all('tr'):
        value = float(row.find('td', {'class':'chart-prize-col'}).text[1:].replace(',',''))
        winner = row.find('td', {'class':'chart-winners-col'}).text
        values[winner] = value
    
    # getting a date
    current_date = soup.find('p', {'class':'large-date'}).text
    
    ### saving
    data['numbers'] = [int(a) for a in win_numbers[:6]]
    data['bonus'] = int(win_numbers[7])
    data['date'] = current_date
    data['money'] = values
    return data

def total_sum(database, info):
    total = 0
    free_wins = 0
    for numbers in database:
        inters = set(numbers).intersection(set(info['numbers']))
        size = len(inters)
        bonus = info['bonus'] in set(numbers)
        win = 0.0
        free_win = 0
        if (size, bonus) == (3, False):
            free_win = 1
            free_wins += 1
        if (size, bonus) in intersections:
            key = intersections[(size, bonus)]
            win = info['money'][key]
        total += win
        print('You win ${} and {} free win for a ticket with numbers {}'.format(win, free_win, numbers))
    print('\nYour total win on {} is ${} and {} free wins'.format(info['date'], total, free_wins))

In [261]:
# Put yor date here:
year = 2019
month = 4
day = 26

info = get_data(get_url(year, month, day))

print('Your tickets are: ')
database = np.loadtxt(filename, dtype=np.int64)
print(database)
print('\n\n')
print('On {} \nwinning numbers are: {} - {}'.format(info['date'], info['numbers'], info['bonus']))
print('Table of winnings:')
for key in info['money']:
    print('{:>15} ----- ${}'.format(key, info['money'][key]))

print()
total_sum(database, info)

Your tickets are: 
[[11 43 34 35 23 21 19]
 [ 1 24  4  3  7 31 17]
 [34  1  2  3  4  5  6]
 [ 2 20 21 30 31 24 34]
 [ 9 17 21 23 36 41 44]]



On Fri, Apr 26 
winning numbers are: [9, 17, 21, 23, 36, 41] - 12
Table of winnings:
            7/7 ----- $35503620.4
    6/7 + Bonus ----- $117240.2
            6/7 ----- $6234.2
            5/7 ----- $107.8
            4/7 ----- $20.0
    3/7 + Bonus ----- $20.0

You win $0.0 and 0 free win for a ticket with numbers [11 43 34 35 23 21 19]
You win $0.0 and 0 free win for a ticket with numbers [ 1 24  4  3  7 31 17]
You win $0.0 and 0 free win for a ticket with numbers [34  1  2  3  4  5  6]
You win $0.0 and 0 free win for a ticket with numbers [ 2 20 21 30 31 24 34]
You win $6234.2 and 0 free win for a ticket with numbers [ 9 17 21 23 36 41 44]

Your total win on Fri, Apr 26 is $6234.2 and 0 free wins
